# 🔍 ดึงข้อมูลหุ้นกลุ่ม SET100 จาก Yahoo Finance + SET.or.th

In [2]:
!pip install yfinance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ------------------------------------- 949.2/949.2 kB 12.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 14.2 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15618 sha256=3e8a3e96eef5916dc3806ea019ba249c506cfb2e76ab24ce06bb52ac52a0c581
  Stored in directory: c:\users\a0864\appdata\local\pip\cache\wheels\1e\df\0f\e2bbb22d689b30c681feb5410ab6

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [3]:
import requests
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

# 1️⃣ ดึงรายชื่อหุ้นในกลุ่ม SET100 จากเว็บ SET.or.th

In [7]:
set100_url = "https://www.set.or.th/en/market/index/set100"
res = requests.get(set100_url)
soup = BeautifulSoup(res.text, "html.parser")

In [8]:
# หา symbol จากตารางในหน้าเว็บ
symbols = []
for tag in soup.select("table.table-info tbody tr td:first-child"):
    sym = tag.text.strip()
    if sym:
        symbols.append(sym)

print(f"✅ พบหุ้นใน SET100 จำนวน {len(symbols)} ตัว")

✅ พบหุ้นใน SET100 จำนวน 0 ตัว


In [6]:
import requests
import pandas as pd

# ✅ ดึงรายชื่อหุ้นใน SET100 จาก settrade.com
url = "https://www.settrade.com/api/set/index/SET100/constituents"
r = requests.get(url).json()

symbols = [item["symbol"] for item in r["data"]]
print(f"✅ พบหุ้นใน SET100 จำนวน {len(symbols)} ตัว")
print(symbols[:10])  # แสดงตัวอย่าง 10 ตัวแรก


JSONDecodeError: Expecting value: line 1 column 1 (char 0)